In [458]:
## 
import pandas as pd
import datetime

import paramiko
import io
import os 
import numpy as np

from snowflake.connector.pandas_tools import pd_writer
from snowflake.connector.pandas_tools import write_pandas
from snowflake.sqlalchemy import URL
from sqlalchemy import create_engine
from slack_sdk import WebClient
from slack_sdk.errors import SlackApiError

from xbbg import blp
import pandas as pd
from dateutil.relativedelta import relativedelta

import requests
import json
import xmltodict


In [459]:
engine = create_engine(URL(
        account = 'QCOHCUV-MM41421',
        user = 'SHATTHIK',
        password = 'Shattiker1',
        database = 'PORT',
        schema = 'CASH',
        warehouse = 'COMPUTE_WH'
    ))

Username=r'NorselabAPI'
Password=r'N0rseL@bAS2022'


In [460]:
BB_PORT = pd.read_sql("SELECT AS_OF_DATE,SECURITY_ID,TOTAL_UNITS FROM PORT.CASH.PORTFOLIO WHERE AS_OF_DATE = '{0}'".format(str(datetime.date.today())),con =engine)

In [461]:
#### SPLIT_POS

TRADES = pd.read_sql("SELECT * FROM TRADE.FLOW.TRADE_APPROVED_FX WHERE settlement_date >= '{0}' or settlement_date_spot >= '{0}' or settlement_date_fwd >= '{0}'".format(str(datetime.date.today())),con =engine)


BOND_TRADES = TRADES[TRADES["contract_type"] == "Bond"].copy()
SPOT_TRADES = TRADES[(TRADES["contract_type"] == "SPOT") | (TRADES["contract_type"] == "SWAP")].copy()
FWD_TRADES = TRADES[(TRADES["contract_type"] == "FWD")|(TRADES["contract_type"] == "SWAP")].copy()

SPOT_TRADES = SPOT_TRADES[SPOT_TRADES.settlement_date_spot>= datetime.date.today()]

######

In [462]:
TRADES

,trade_date,trader,settlement_date,settlement_date_spot,settlement_date_fwd,broker,instrument,contract_type,trade_type,trade_type_spot,...,gross,accured,accured_days,nett,comment,status,status_sentrbc,status_ftp,refid,created
0,2022-11-22,TH,2022-11-24,None,None,CLA,Bond,Bond,BUY,None,...,13950000.0,67308.75,27.0,14017308.75,Nothing\n,APPROVED,NO,PENDING,CLA5262211221122,2022-11-22 11:22:53
1,2022-11-22,TH,2022-11-24,None,None,PAR,Bond,Bond,BUY,None,...,7070000.0,100100.00,52.0,7170100.00,Nothing\n,APPROVED,NO,PENDING,PAR1652211221125,2022-11-22 11:25:28
2,2022-11-22,TH,2022-11-24,None,None,FEA,Bond,Bond,BUY,None,...,2150000.0,51458.33,156.0,2201458.33,Nothing\n,APPROVED,NO,PENDING,FEA8762211221126,2022-11-22 11:26:57
3,2022-11-22,TH,2022-11-24,None,None,DAN,Bond,Bond,BUY,None,...,3920000.0,16071.87,11.0,3936071.87,Nothing\n,APPROVED,NO,PENDING,DAN9872211221128,2022-11-22 11:28:21
4,2022-11-22,TH,None,2022-11-24,2023-03-31,RBC,FX,SWAP,None,BUY,...,NaN,NaN,NaN,NaN,None,APPROVED,NO,PENDING,RBCEN11221339,2022-11-22 13:39:15
5,2022-11-22,TH,None,2022-11-24,2023-03-31,RBC,FX,SWAP,None,BUY,...,NaN,NaN,NaN,NaN,None,APPROVED,NO,PENDING,RBCSN11221342,2022-11-22 13:42:07


In [463]:
BB_PORT

,as_of_date,security_id,total_units
0,2022-11-22,EUR,0.0
1,2022-11-22,NOK,500000000.0
2,2022-11-22,SEK,0.0
3,2022-11-22,USD,0.0
4,2022-11-22,DKK,0.0


In [464]:
def BOND_TRADE(BOND_TRADES,BB_PORT):
    BB_PORT = BB_PORT.copy()

    BOND_TRADES = BOND_TRADES[["trade_date","settlement_date","trade_type","isin","amount","nett","cur"]].copy()
    BOND_TRADES.loc[BOND_TRADES["trade_type"] == "SELL","amount"] = BOND_TRADES.loc[BOND_TRADES["trade_type"] == "SELL","amount"]*-1
    BOND_TRADES.loc[BOND_TRADES["trade_type"] == "BUY","nett"] = BOND_TRADES.loc[BOND_TRADES["trade_type"] == "BUY","nett"]*-1
    BOND_TRADES_agg = BOND_TRADES.groupby(["isin","cur"]).agg({"amount":"sum","nett":"sum"}).reset_index()


    ##BOND


    for index, row in BOND_TRADES_agg.iterrows():
        if row["isin"] in list(BB_PORT["security_id"]):
            BB_PORT.loc[BB_PORT["security_id"] == row["isin"],"total_units"] = row["amount"]+BB_PORT.loc[BB_PORT["security_id"] == row["isin"],"total_units"]
        else:
            df_app = pd.DataFrame([{'as_of_date': datetime.date.today(),
        'security_id': row["isin"],
        'total_units': row["amount"]}])
            BB_PORT = pd.concat([BB_PORT,df_app])

        BB_PORT.loc[BB_PORT["security_id"] == row["cur"],"total_units"] = row["nett"]+BB_PORT.loc[BB_PORT["security_id"] == row["cur"],"total_units"]

    return BB_PORT

    


In [465]:
BB_PORT_1 = BOND_TRADE(BOND_TRADES,BB_PORT)

In [466]:
BB_PORT_1

,as_of_date,security_id,total_units
0,2022-11-22,EUR,-2.201458e+06
1,2022-11-22,NOK,4.928299e+08
2,2022-11-22,SEK,-1.401731e+07
3,2022-11-22,USD,-3.936072e+06
4,2022-11-22,DKK,0.000000e+00
0,2022-11-22,FI4000507876,2.500000e+06
0,2022-11-22,NO0010991987,4.000000e+06
0,2022-11-22,NO0012708165,7.000000e+06
0,2022-11-22,SE0016844526,1.500000e+07


In [467]:
def SPOT_TRADE(SPOT_TRADES,BB_PORT):
    ###FX SPOT
    BB_PORT = BB_PORT.copy()

    SPOT_TRADES = SPOT_TRADES[["trade_date","settlement_date_spot","buy_cur_spot","sell_cur_spot","buy_amount_spot","sell_amount_spot"]].copy()
    SPOT_TRADES["sell_amount_spot"] = SPOT_TRADES["sell_amount_spot"]*-1


    spot_buyleg = SPOT_TRADES[["buy_cur_spot","buy_amount_spot"]]
    spot_sellleg = SPOT_TRADES[["sell_cur_spot","sell_amount_spot"]]

    spot_buyleg.columns = spot_sellleg.columns = ["isin","nett"]

    spot_trans = pd.concat([spot_buyleg,spot_sellleg]).groupby(["isin"]).nett.sum().reset_index()

    for index, row in spot_trans.iterrows():
        BB_PORT.loc[BB_PORT["security_id"] == row["isin"],"total_units"] = row["nett"]+BB_PORT.loc[BB_PORT["security_id"] == row["isin"],"total_units"]

    return BB_PORT

In [468]:
BB_PORT_2 = SPOT_TRADE(SPOT_TRADES,BB_PORT_1)

In [469]:
BB_PORT_2

,as_of_date,security_id,total_units
0,2022-11-22,EUR,0.000000e+00
1,2022-11-22,NOK,4.603317e+08
2,2022-11-22,SEK,-4.017309e+06
3,2022-11-22,USD,-3.936072e+06
4,2022-11-22,DKK,0.000000e+00
0,2022-11-22,FI4000507876,2.500000e+06
0,2022-11-22,NO0010991987,4.000000e+06
0,2022-11-22,NO0012708165,7.000000e+06
0,2022-11-22,SE0016844526,1.500000e+07


In [470]:
###FX FWD
REF_RATES = {"NOK":'YCMM0044 Index',"EUR":'YCMM0085 Index',"USD":'YCMM0021 Index',"SEK":"YCMM0045 Index","DKK":"YCMM0042 Index"}
IBOR_CURVE = blp.bds(tickers=REF_RATES["NOK"], flds=['CURVE_TENOR_RATES'])

def diff_month(start, end):
    return 360*(end.year - start.year)\
               + 30*(end.month - start.month -1)\
               + max(0, 30 - start.day)\
               + min(30, end.day)

def M2M(trade,IBOR_CURVE):

    value_date = trade["settlement_date_fwd"]
    
    if trade['buy_cur_fwd'] == "NOK":
        BASE = trade['sell_cur_fwd']
        NOK_K = trade['buy_cur_fwd']
        ttype = "SELL"
        f0 = trade["buy_amount_fwd"]/trade["sell_amount_fwd"]
        amt = trade["buy_amount_fwd"]
        qty = amt/f0
        
    else: 
        BASE = trade['buy_cur_fwd']
        NOK_K = trade['sell_cur_fwd']
        ttype = "BUY"
        f0 = trade["sell_amount_fwd"]/trade["buy_amount_fwd"]
        amt = trade["sell_amount_fwd"]
        qty = amt/f0

    

    

    def addDay(td,ad):
        if ad[-1] == "W":
            dv = relativedelta(weeks=+int(ad[0:-1])) + datetime.date.today()
        if ad[-1] == "M":
            dv = relativedelta(months=+int(ad[0:-1])) + datetime.date.today()
        if ad[-1] == "Y":
            dv = relativedelta(years=+int(ad[0:-1])) + datetime.date.today()

        return(dv)

    ticker_fwd = '{0:}/{1:} {2:} Curncy'.format(BASE,NOK_K,value_date.strftime('%m/%d/%y'))

    fwd = blp.bdp(ticker_fwd,["PX_BID",'PX_ASK'])



    ticker = '{0:}/{1:} Curncy'.format(BASE,NOK_K)
    spot = blp.bdp(ticker,["PX_BID",'PX_ASK'])

    
    IBOR_CURVE["tenor_dates"] = IBOR_CURVE.tenor.apply(lambda x: addDay(IBOR_CURVE.last_update.iloc[0],x))
    daily_rates = IBOR_CURVE[["tenor_dates","mid_yield"]].set_index("tenor_dates").reindex(pd.date_range(start=IBOR_CURVE.tenor_dates.min(), end=IBOR_CURVE.tenor_dates.max(), freq='1D')).interpolate(method='linear') 
    daily_rates.index.freq=None  

    
    nday = diff_month(pd.to_datetime(value_date),pd.to_datetime(trade["settlement_date_fwd"]))

    if ttype == "SELL":
        PL = ((f0-fwd[{"SELL":"px_bid","BUY":"px_ask"}[ttype]].iloc[0])*qty)/(1+daily_rates.loc[pd.to_datetime(value_date)].iloc[0]/100*(nday/360))
    if ttype == "BUY":
        PL = ((fwd[{"SELL":"px_bid","BUY":"px_ask"}[ttype]].iloc[0]-f0)*qty)/(1+daily_rates.loc[pd.to_datetime(value_date)].iloc[0]/100*(nday/360))


    r_d = {'as_of_date': datetime.date.today(),
    'security_id': f'{trade["buy_cur_fwd"]}/{trade["sell_cur_fwd"]}',
    'total_units': PL}

    #print(ticker_fwd)

    return r_d


def FWD_TRADE(FWD_TRADES,BB_PORT):
    BB_PORT = BB_PORT.copy()

    fwd_buyleg = FWD_TRADES[["buy_cur_fwd","buy_amount_fwd"]].copy()
    fwd_sellleg = FWD_TRADES[["sell_cur_fwd","sell_amount_fwd"]].copy()
    fwd_sellleg["sell_amount_fwd"] = fwd_sellleg["sell_amount_fwd"]*-1


    fwd_buyleg.columns = fwd_sellleg.columns = ["isin","nett"]
    fwd_trans = pd.concat([fwd_buyleg,fwd_sellleg]).groupby(["isin"]).nett.sum().reset_index()


    if fwd_trans.shape[0]>0:

        fwd_pos = []
        for i in range(0,FWD_TRADES.shape[0]):
            fwd_pos.append(M2M(FWD_TRADES.iloc[i],IBOR_CURVE))

        FWD_LINES = pd.DataFrame(fwd_pos)
        FWD_LINES= FWD_LINES.groupby(["as_of_date","security_id"]).total_units.sum().reset_index()
    else:
        FWD_LINES = pd.DataFrame(columns=["as_of_date","security_id","total_units"])


    BB_PORT = pd.concat([BB_PORT,FWD_LINES])
    BB_PORT = BB_PORT.groupby(["as_of_date","security_id"]).total_units.sum().reset_index()

    return BB_PORT



In [471]:
BB_PORT_3 = FWD_TRADE(FWD_TRADES,BB_PORT_2)

In [472]:
BB_PORT_3

,as_of_date,security_id,total_units
0,2022-11-22,DKK,0.000000e+00
1,2022-11-22,EUR,0.000000e+00
2,2022-11-22,FI4000507876,2.500000e+06
3,2022-11-22,NO0010991987,4.000000e+06
4,2022-11-22,NO0012708165,7.000000e+06
5,2022-11-22,NOK,4.603317e+08
6,2022-11-22,NOK/EUR,-8.776334e+04
7,2022-11-22,NOK/SEK,-5.758920e+04
8,2022-11-22,SE0016844526,1.500000e+07
9,2022-11-22,SEK,-4.017309e+06


In [473]:
BB_PORT_3.to_excel("port_bb.xlsx",index=False)



hostname = "sftp.bloomberg.com"
ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy( paramiko.AutoAddPolicy() )
ssh.connect(hostname, port = 22 ,username="u1251988031", password="BloombergNorselab1=")


ftp = ssh.open_sftp()
files = ftp.listdir()
ftp.put("port_bb.xlsx", "port_bb.xlsx")
ssh.close()

In [474]:
BB_PORT_3

,as_of_date,security_id,total_units
0,2022-11-22,DKK,0.000000e+00
1,2022-11-22,EUR,0.000000e+00
2,2022-11-22,FI4000507876,2.500000e+06
3,2022-11-22,NO0010991987,4.000000e+06
4,2022-11-22,NO0012708165,7.000000e+06
5,2022-11-22,NOK,4.603317e+08
6,2022-11-22,NOK/EUR,-8.776334e+04
7,2022-11-22,NOK/SEK,-5.758920e+04
8,2022-11-22,SE0016844526,1.500000e+07
9,2022-11-22,SEK,-4.017309e+06


In [475]:
engine = create_engine(URL(
        account = 'QCOHCUV-MM41421',
        user = 'SHATTHIK',
        password = 'Shattiker1',
        database = 'PORT',
        schema = 'CASH',
        warehouse = 'COMPUTE_WH'
    ))

In [476]:
##
def callstamdata(Username,Password,bdy):
    url = "https://feed.stamdata.com/services/IStamdataDataService/getDataForIdentifiers" 

    

    headers = {'Content-type': 'application/json', 'Accept': 'text/plain'}

    res = requests.post(
    url, 
    auth=(Username, Password),
    data  = json.dumps(bdy),headers=headers
    )

    obj   =xmltodict.parse(res.json())

    return obj

def get_intday(yr,dd):

    
    d = dd.split("/")[1].upper()

    if "360" in d:
         day_dem = 360
         
    if "365" in d:
         day_dem = 365

    if "ACT" in d:
         day_dem = pd.Timestamp(yr.year, 12, 31).dayofyear


    return day_dem




SECS = BB_PORT_3[BB_PORT_3.security_id.apply(lambda x: len(x) == 12)]


bdy = [{"PrimaryKey":i,"table": 0}for i in SECS.security_id]



In [477]:
#####CHASH LEDGER
BOND_TRADES_CL = BOND_TRADES.copy()
BOND_TRADES_CL.loc[BOND_TRADES_CL["trade_type"] == "BUY","nett"] = BOND_TRADES_CL.loc[BOND_TRADES_CL["trade_type"] == "BUY","nett"]*-1
BOND_CL = BOND_TRADES_CL.groupby(["cur","settlement_date"]).nett.sum().reset_index()
port_dt = pd.read_sql("SELECT SECURITY_ID,AS_OF_DATE,TOTAL_UNITS FROM PORT.CASH.PORTFOLIO WHERE AS_OF_DATE = '{0}' AND ASSET_CATEGORY = 'FX'".format(str(datetime.date.today())),con =engine)
port_dt.columns = ["cur","settlement_date","nett"]

###cuopons

BOND_TRADES_sett = BOND_TRADES_CL.groupby(["isin"]).settlement_date.min().to_dict()



In [478]:
BOND_CL

,cur,settlement_date,nett
0,EUR,2022-11-24,-2201458.33
1,NOK,2022-11-24,-7170100.00
2,SEK,2022-11-24,-14017308.75
3,USD,2022-11-24,-3936071.87


In [479]:
if SECS.shape[0] > 0:

    obj = callstamdata(Username,Password,bdy)

    All_intpay = []

    for bd in obj["database"]["table_Issue"]["Issue"]:

        MaturityPrice = float(bd["MaturityPrice"])

        CUP = pd.DataFrame(bd["table_IssueTermDate"]["IssueTermDate"])
        CUP["InterestCoupon_20160302"] = CUP["InterestCoupon_20160302"].ffill()

        CUP["total_units"] = SECS[SECS.security_id == bd["ISIN"]]["total_units"].iloc[0]
        CUP["days_denom"] = [get_intday(i,j) for i,j in zip(pd.to_datetime(CUP.TermDate),CUP.AccruedInterestDayCountConvention)]

        CUP["int_pay"] = CUP["total_units"]*((CUP["InterestCoupon_20160302"].astype(float))/100)*CUP["DaysInPeriod"].astype(int)/CUP["days_denom"].astype(int)
        CUP["red_pay"] = 0

        CUP.loc[CUP.Installment.astype(float) > 0,"red_pay"] = CUP.loc[CUP.Installment.astype(float) > 0,"total_units"]*(MaturityPrice/100)

        CUP["pay"]= CUP["int_pay"]+CUP["red_pay"]
        CUP["Currency"] = bd["Currency"]

        CUP = CUP[pd.to_datetime(CUP.TermDate) >= pd.to_datetime(BOND_TRADES_sett.get(bd["ISIN"],BB_PORT["as_of_date"].min()))]

        

        All_intpay.append(CUP[["Currency","TermDate","pay"]])

    INT_TRADES = pd.concat(All_intpay).groupby(["Currency","TermDate"]).sum().reset_index()
    INT_TRADES.columns = ["cur","settlement_date","nett"]
else:
    INT_TRADES = pd.DataFrame(columns= ["cur","settlement_date","nett"])

In [480]:
INT_TRADES

,cur,settlement_date,nett
0,EUR,2022-12-18,5.937500e+04
1,EUR,2023-06-18,5.937500e+04
2,EUR,2023-12-18,5.937500e+04
3,EUR,2024-06-18,5.937500e+04
4,EUR,2024-12-18,5.937500e+04
5,EUR,2025-06-18,5.937500e+04
6,EUR,2025-12-18,5.937500e+04
7,EUR,2026-06-18,2.559375e+06
8,NOK,2023-01-03,1.771000e+05
9,NOK,2023-04-03,1.732500e+05


In [481]:
SPOT_TRADES_CL = SPOT_TRADES.copy()
SPOT_TRADES_CL["sell_amount_spot"] = SPOT_TRADES_CL["sell_amount_spot"]*-1
SPOT_TRADES_BUY = SPOT_TRADES_CL[["buy_cur_spot","settlement_date_spot","buy_amount_spot"]].copy()
SPOT_TRADES_SELL = SPOT_TRADES_CL[["sell_cur_spot","settlement_date_spot","sell_amount_spot"]].copy()



SPOT_TRADES_BUY.columns = SPOT_TRADES_SELL.columns = ["cur","settlement_date","nett"]

SPOT_CL = pd.concat([SPOT_TRADES_BUY,SPOT_TRADES_SELL]).groupby(["cur","settlement_date"]).nett.sum().reset_index()

In [482]:
SPOT_CL

,cur,settlement_date,nett
0,EUR,2022-11-24,2201458.33
1,NOK,2022-11-24,-32498224.97
2,SEK,2022-11-24,10000000.00


In [483]:
FWD_TRADES_BUY = FWD_TRADES[["buy_cur_fwd","settlement_date_fwd","buy_amount_fwd"]].copy()
FWD_TRADES_SELL = FWD_TRADES[["sell_cur_fwd","settlement_date_fwd","sell_amount_fwd"]].copy()
FWD_TRADES_SELL["sell_amount_fwd"] = FWD_TRADES_SELL["sell_amount_fwd"]*-1

FWD_TRADES_BUY.columns = FWD_TRADES_SELL.columns = ["cur","settlement_date","nett"]

FWD_CL = pd.concat([FWD_TRADES_BUY,FWD_TRADES_SELL]).groupby(["cur","settlement_date"]).nett.sum().reset_index()

In [484]:
FWD_CL

,cur,settlement_date,nett
0,EUR,2023-03-31,-2201458.33
1,NOK,2023-03-31,32527254.13
2,SEK,2023-03-31,-10000000.00


In [568]:

ALL_CL = pd.concat([BOND_CL,SPOT_CL,FWD_CL,port_dt,INT_TRADES]).groupby(["cur","settlement_date"]).nett.sum().reset_index()


tickers = list(set(['{0:}/NOK Curncy'.format(i) for i in ALL_CL.cur]))
spot = blp.bdp(tickers,["PX_BID","PX_MID",'PX_ASK'])
spot["cur"] =  [i.split("/")[0] for i in spot.index]

cur_dict = spot[["cur","px_mid"]].copy().set_index("cur").to_dict()["px_mid"]

total_cl = []
for index, row in ALL_CL.iterrows():
   total_cl.append(cur_dict.get(row["cur"])*row["nett"])


ALL_CL["total"] = total_cl

CASH_LEDGER = ALL_CL.pivot(index="cur",columns="settlement_date",values="nett").fillna(0)
CASH_LEDGER_tot = ALL_CL.pivot(index="cur",columns="settlement_date",values="total").fillna(0)
CASH_LEDGER.loc["Total"] = CASH_LEDGER_tot.sum(0)


CASH_LEDGER.columns = pd.to_datetime(CASH_LEDGER.columns)
CASH_LEDGER = CASH_LEDGER[CASH_LEDGER.columns.sort_values()]
CASH_LEDGER.columns = [i.date() for i in CASH_LEDGER.columns]

CASH_LEDGER = CASH_LEDGER.cumsum(1).reset_index()
CASH_LEDGER.columns = [str(i).upper() for i in CASH_LEDGER.columns]


In [569]:
CASH_LEDGER.to_sql("cashledger",con = engine,if_exists="replace",index=False,method=pd_writer)

In [566]:
CASH_LEDGER

,CUR,2022-11-22,2022-11-24,2022-12-18,2023-01-03,2023-01-30,2023-02-13,2023-03-31,2023-04-03,2023-04-28,...,2024-12-18,2025-01-03,2025-02-12,2025-04-03,2025-05-12,2025-06-18,2025-07-03,2025-10-03,2025-12-18,2026-06-18
0,DKK,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
1,EUR,0.0,0.000000e+00,5.937500e+04,5.937500e+04,5.937500e+04,5.937500e+04,-2.142083e+06,-2.142083e+06,-2.142083e+06,...,-1.904583e+06,-1.904583e+06,-1.904583e+06,-1.904583e+06,-1.904583e+06,-1.845208e+06,-1.845208e+06,-1.845208e+06,-1.785833e+06,7.735417e+05
2,NOK,500000000.0,4.603317e+08,4.603317e+08,4.605088e+08,4.605088e+08,4.605088e+08,4.930360e+08,4.932093e+08,4.932093e+08,...,4.942661e+08,4.944432e+08,4.944432e+08,4.946165e+08,4.946165e+08,4.946165e+08,4.947916e+08,5.019687e+08,5.019687e+08,5.019687e+08
3,SEK,0.0,-4.017309e+06,-4.017309e+06,-4.017309e+06,-3.782975e+06,-3.782975e+06,-1.378297e+07,-1.378297e+07,-1.356360e+07,...,2.805013e+06,2.805013e+06,2.805013e+06,2.805013e+06,2.805013e+06,2.805013e+06,2.805013e+06,2.805013e+06,2.805013e+06,2.805013e+06
4,USD,0.0,-3.936072e+06,-3.936072e+06,-3.936072e+06,-3.936072e+06,-3.803114e+06,-3.803114e+06,-3.803114e+06,-3.803114e+06,...,-2.870945e+06,-2.870945e+06,-2.736526e+06,-2.736526e+06,1.393510e+06,1.393510e+06,1.393510e+06,1.393510e+06,1.393510e+06,1.393510e+06
5,Total,500000000.0,4.164058e+08,4.170262e+08,4.172033e+08,4.174263e+08,4.187810e+08,4.187890e+08,4.189623e+08,4.191710e+08,...,4.477797e+08,4.479568e+08,4.493264e+08,4.494997e+08,4.915800e+08,4.922005e+08,4.923756e+08,4.995527e+08,5.001732e+08,5.269192e+08


In [401]:
###push to database

In [402]:
def BOND_TRADE_hd(BOND_TRADES,BB_PORT):
    BB_PORT = BB_PORT.copy()

    BOND_TRADES = BOND_TRADES[["trade_date","settlement_date","trade_type","isin","amount","nett","cur"]].copy()
    BOND_TRADES.loc[BOND_TRADES["trade_type"] == "SELL","amount"] = BOND_TRADES.loc[BOND_TRADES["trade_type"] == "SELL","amount"]*-1
    BOND_TRADES.loc[BOND_TRADES["trade_type"] == "BUY","nett"] = BOND_TRADES.loc[BOND_TRADES["trade_type"] == "BUY","nett"]*-1
    BOND_TRADES_agg = BOND_TRADES.groupby(["isin","cur"]).agg({"amount":"sum","nett":"sum"}).reset_index()


    ##BOND


    for index, row in BOND_TRADES_agg.iterrows():
        if row["isin"] in list(BB_PORT["security_id"]):
            BB_PORT.loc[BB_PORT["security_id"] == row["isin"],"total_units"] = row["amount"]+BB_PORT.loc[BB_PORT["security_id"] == row["isin"],"total_units"]
        else:
            df_app = pd.DataFrame([{'as_of_date': datetime.date.today(),
        'security_id': row["isin"],
        'total_units': row["amount"]}])
            BB_PORT = pd.concat([BB_PORT,df_app])

        #BB_PORT.loc[BB_PORT["security_id"] == row["cur"],"total_units"] = row["nett"]+BB_PORT.loc[BB_PORT["security_id"] == row["cur"],"total_units"]

    return BB_PORT

    


In [403]:
###hedge###
BB_HD = BOND_TRADE_hd(BOND_TRADES,BB_PORT)
HEDGE_PORT = BB_HD[BB_HD.security_id.apply(lambda x:len(x)!= 7)].copy()

bd_isin = list(HEDGE_PORT[HEDGE_PORT.security_id.apply(lambda x:len(x)== 12)].security_id)
bd_isin = ["/ISIN/"+i for i in bd_isin]

if len(bd_isin) > 0:
    bd_price = blp.bdp(bd_isin,["CRNCY","PX_DIRTY_BID"])
    bd_price["security_id"] = [i.split("/")[2]for i in bd_price.index]
else:
    bd_price = pd.DataFrame([{"crncy":"NOK","px_dirty_bid":100,"security_id":"NOK"}])
HEDGE_PORT = HEDGE_PORT.merge(bd_price,on = "security_id",how ="left")
HEDGE_PORT.crncy = HEDGE_PORT.crncy.fillna(HEDGE_PORT.security_id)
HEDGE_PORT.px_dirty_bid = HEDGE_PORT.px_dirty_bid.fillna(100)
HEDGE_PORT["mval"] = HEDGE_PORT["total_units"]*HEDGE_PORT["px_dirty_bid"]/100
PORT_EXP = HEDGE_PORT.groupby("crncy").mval.sum().reset_index()



FWD_TRADES_BUY = FWD_TRADES[["buy_cur_fwd","settlement_date_fwd","buy_amount_fwd"]].copy()
FWD_TRADES_SELL = FWD_TRADES[["sell_cur_fwd","settlement_date_fwd","sell_amount_fwd"]].copy()
FWD_TRADES_SELL["sell_amount_fwd"] = FWD_TRADES_SELL["sell_amount_fwd"]*-1
FWD_TRADES_BUY.columns = FWD_TRADES_SELL.columns = ["cur","settlement_date","nett"]
FWD_CL = pd.concat([FWD_TRADES_BUY,FWD_TRADES_SELL]).groupby(["cur","settlement_date"]).nett.sum().reset_index()

if FWD_CL.shape[0]>0:

    HEDGE_FWD = FWD_CL.groupby("cur").sum("nett").reset_index()
else:
    HEDGE_FWD = pd.DataFrame(columns=["cur","nett"])




HEDGE_FWD.columns = ["crncy","fwd"]
TOTAL_HEDGE = PORT_EXP.merge(HEDGE_FWD,on ="crncy",how ="outer")
TOTAL_HEDGE = TOTAL_HEDGE[TOTAL_HEDGE.crncy != "NOK"].fillna(0)
TOTAL_HEDGE["HEDGE_RATIO"] =  1-(TOTAL_HEDGE["mval"]+TOTAL_HEDGE["fwd"])/TOTAL_HEDGE["mval"]
TOTAL_HEDGE["HEDGE_REST"] = (TOTAL_HEDGE["mval"]+TOTAL_HEDGE["fwd"])*-1
TOTAL_HEDGE["DATE"] = str(datetime.date.today())
TOTAL_HEDGE.columns = [str(i).upper() for i in TOTAL_HEDGE.columns]
TOTAL_HEDGE = TOTAL_HEDGE.fillna(0)

In [404]:

TOTAL_HEDGE.to_sql("hedge",con = engine,if_exists="replace",index=False,method=pd_writer)

In [405]:
TOTAL_HEDGE

,CRNCY,MVAL,FWD,HEDGE_RATIO,HEDGE_REST,DATE
0,DKK,0.000000e+00,0.00,0.000000,-0.000000e+00,2022-11-22
1,EUR,2.163433e+06,-2201458.33,1.017576,3.802500e+04,2022-11-22
3,SEK,1.385486e+07,-10000000.00,0.721768,-3.854859e+06,2022-11-22
4,USD,3.968392e+06,0.00,0.000000,-3.968392e+06,2022-11-22
